In [7]:
from tbparse import SummaryReader
import polars as pl

In [8]:
logs = SummaryReader("runs/", pivot=True, extra_columns={'dir_name'})

In [9]:
data = pl.from_pandas(logs.hparams, nan_to_null=False)

ArrowInvalid: Could not convert 'None' with type str: tried to convert to double

In [13]:
logs.hparams

,activation,batch_size,beats,causal,center,classifier_dim,context_size,d_conv,d_state,dataset_version,...,sample_rate,scheduler,segment_type,splits,test_batch_size,time_shift,train_set,use_relative_pos,weight_decay,dir_name
0,SELU,512.0,0.0,0.0,1.0,64.0,NaN,NaN,NaN,L,...,44100.0,1.0,NaN,"(0.85, 0.15, 0.0)",NaN,0.000,NaN,NaN,NaN,Jun01_12-27-57_seppel-liemarce/1717268405.8346386
1,SELU,512.0,0.0,0.0,1.0,256.0,NaN,NaN,NaN,L,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.000,all,NaN,NaN,Jun01_21-17-49_seppel-liemarce/1717305602.794235
2,SELU,512.0,0.0,1.0,1.0,128.0,NaN,NaN,NaN,L,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.015,all,NaN,NaN,Jun02_10-04-23_seppel-liemarce/1717344383.2908068
3,SELU,512.0,0.0,1.0,1.0,128.0,NaN,NaN,NaN,L,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.025,a2md_train,NaN,NaN,Jun02_16-16-02_seppel-liemarce/1717355029.714263
4,SELU,512.0,0.0,1.0,1.0,256.0,NaN,NaN,NaN,S,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.025,a2md_train,NaN,NaN,Jun02_21-13-42_seppel-liemarce/1717362873.1208162
5,SELU,512.0,0.0,1.0,1.0,256.0,NaN,NaN,NaN,S,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.025,a2md_train,NaN,NaN,Jun02_21-17-37_seppel-liemarce/1717363107.7022955
6,SELU,512.0,0.0,1.0,1.0,256.0,NaN,NaN,NaN,S,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.025,a2md_train,NaN,NaN,Jun02_21-24-44_seppel-liemarce/1717363535.1824746
7,SELU,512.0,0.0,1.0,1.0,256.0,NaN,NaN,NaN,S,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.025,a2md_train,NaN,NaN,Jun02_21-26-56_seppel-liemarce/1717363876.9544137
8,SELU,512.0,0.0,1.0,1.0,256.0,NaN,NaN,NaN,L,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.025,a2md_train,NaN,NaN,Jun02_21-32-36_seppel-liemarce/1717375708.1614242
9,SELU,512.0,0.0,1.0,1.0,64.0,NaN,NaN,NaN,L,...,44100.0,1.0,frame,"(0.85, 0.15, 0.0)",NaN,0.015,a2md_train,NaN,NaN,Jun03_11-18-29_seppel-liemarce/1717423908.1089978
